In [1]:
import pyspark
from pyspark.sql import SparkSession

print("pyspark version:", pyspark.__version__)

pyspark version: 4.1.1


In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/22 15:31:43 WARN Utils: Your hostname, Robs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.197 instead (on interface en0)
26/02/22 15:31:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/22 15:31:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.option("recursiveFileLookup", "true").parquet("data/pq/green")

```sql
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [4]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [5]:
from datetime import datetime

In [6]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [7]:
rows = rdd.take(10)
row = rows[0]

In [8]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 16, 23, 31, 44), PULocationID=65, total_amount=25.8)

In [9]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [10]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [11]:
from collections import namedtuple

In [12]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [13]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [14]:
from pyspark.sql import types

In [15]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [16]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema)

In [17]:
df_result.write.parquet('tmp/green-revenue', mode='overwrite')

26/02/22 15:34:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [18]:
df_result.rdd.getNumPartitions()

8

In [19]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [20]:
import pandas as pd

In [21]:
rows = duration_rdd.take(10)

In [22]:
df = pd.DataFrame(rows, columns=columns)

In [23]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [24]:
#model = ...

def model_predict(df):
    # y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [25]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples(index=False):
        yield tuple(row)

In [26]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch) \
    .toDF(columns + ['predicted_duration'])

In [27]:
df_predicts.printSchema()

root
 |-- VendorID: double (nullable = true)
 |-- lpep_pickup_datetime: struct (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- predicted_duration: double (nullable = true)



In [28]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|              43.0|
|              25.8|
|             81.35|
|               7.1|
|              23.1|
|             13.35|
|               3.2|
|              2.15|
|6.8999999999999995|
|               5.0|
|               7.1|
|41.150000000000006|
|             33.55|
|               0.0|
|             41.25|
|7.1499999999999995|
|              51.2|
|              3.05|
| 8.100000000000001|
|17.150000000000002|
+------------------+
only showing top 20 rows


Traceback (most recent call last):                                              
  File "/Users/rob/Projects/GitHub/data-engineering-zoomcamp/07-batch-processing/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 233, in manager
    code = worker(sock, authenticated)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rob/Projects/GitHub/data-engineering-zoomcamp/07-batch-processing/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 87, in worker
    outfile.flush()
BrokenPipeError: [Errno 32] Broken pipe
